In [2]:
from __future__ import print_function

"""
A script to run the cqed_rhf and cqed_cis method on MgH+ in a cc-pVDZ basis set
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from helper_cis import *
from helper_cs_cqed_cis import *
from psi4.driver.procrouting.response.scf_response import tdscf_excitations
from matplotlib import pyplot as plt
# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2


Methods to fit Pauli-Fierz and Jaynes-Cummings models to CIS data for the ground- and excited-state
of a quantum system (including transition dipole moments in both cases, and the total dipole
moments in the Pauli-Fierz case) given a photon frequency and coupling strength.

# MgH+ Near resonance with photon

Run the block below to compute  CS-CQED-CIS energies and wavefunctions for MgH+ approximately 
resonant with a photon with complex frequency.  The left and right eigenvectors
will be stored to `LCIS` and `RCIS`, respectively, and the Hamiltonian matrix will be stored to 
`H`.  In principle, the Left and Right Eigenvectors are now biorthogonal and satisfy

$$ {\bf L}^{\dagger} \: {\bf R} = {\bf I} $$

in addition to

$$  {\bf L}^{\dagger} \: {\bf H}   = E \: {\bf L}  $$

and 

$$ {\bf H} \: {\bf R} = E  \:{\bf R} $$

In [9]:
# template for z-matrix
molstr = """
Mg
H 1 2.0
symmetry c1
1 1
"""

# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk'
               }

# set psi4 options
psi4.set_options(options_dict)



# electric field strength 
lam_z = 0.0125

# photon energy
om_r = 4.75 / 27.211

# approximate maximum value of transition dipole moment in a.u.
max_tdm = 2.25

# approximate max value of g
g_max = np.sqrt(om_r/2) * lam_z * max_tdm

print(" value of gamma to cross over from weak to strong coupling:", 4 * g_max)
om = om_r -  1 * (g_max/2) * 1j

# lambda vector
lam = np.array([0,0,lam_z])

# loop over the different bond-lengths, create different instances
# of MgH+ molecule and compute and store various quantities
mol = psi4.geometry(molstr)
cqed_dict = cs_cqed_cis(lam, om, molstr, options_dict)
    
# get cqed-rhf energy
cqed_scf_e = cqed_dict['CQED-RHF ENERGY']
# get cqed-cis excitation energies
cqed_cis_e = cqed_dict['CQED-CIS ENERGY']

RCIS = cqed_dict['CQED-CIS R VECTORS']
LCIS = cqed_dict['CQED-CIS L VECTORS']


H = cqed_dict['CQED-CIS H MATRIX']

 

 value of gamma to cross over from weak to strong coupling: 0.033236254686047906

Start SCF iterations:

Canonical RHF One-electron energy = -282.2898820081355211
CQED-RHF One-electron energy      = -282.2901171135010259
Nuclear repulsion energy          = 3.1750632640200003
Dipole energy                     = 0.0008827413911168
SCF Iteration   1: Energy = -199.8812587213680274   dE = -1.99881E+02   dRMS = 2.21975E-05
SCF Iteration   2: Energy = -199.8812595818003501   dE = -8.60432E-07   dRMS = 8.34190E-06
SCF Iteration   3: Energy = -199.8812596865761577   dE = -1.04776E-07   dRMS = 3.21770E-06
SCF Iteration   4: Energy = -199.8812597017974895   dE = -1.52213E-08   dRMS = 1.23677E-06
Total time for SCF iterations: 0.009 seconds 

QED-RHF   energy: -199.88125970 hartree
Psi4  SCF energy: -199.88178387 hartree
using la eig


Now we will confirm that 

$$ {\bf L}^{\dagger} \: {\bf R}  = {\bf I}. $$

Note that ${\bf L}$ and ${\bf R}$ are 2D numpy arrays where the row labels the element of a given eigenvector the column labels the eigenvector itself; e.g. `L[:,0]` is the entire ground-state left eigenvector, `L[:,1]` is the entire first excited-state left eigenvector, etc.  What the biorthogonality of ${\bf L}$ and ${\bf R}$ means
is that the left and right eigenvectors of a given state have an inner product of 1, and the left and right eigenvectors of any two different states have an inner product of 0.

`np.dot( np.conj(L[:,n]), R[:,m] ) -> 1 if n==m and 0 if n!=m`

The following block will evaluate these inner products and print the diagonals and any deveations from 0 inner product of off-diagonals.


In [10]:


dim = len(LCIS[0,:])
from scipy.linalg import lu
from scipy.linalg import inv
M = np.zeros((dim, dim), dtype=complex)
for i in range(0, dim):
    for j in range(0, dim):
        L = np.conj(LCIS[:,i])
        R = RCIS[:,j]
        o = np.dot(L, R)
        M[i,j] = o
        if i==j:
            print(i,j,o)
        elif np.isclose(o, 0+0j, atol=1e-6):
            te = 2+2
        else:
            print(i,j,o)



0 0 (0.9999999999999999+9.255423135529333e-19j)
1 1 (1.0000000000000004+1.3877787807814457e-17j)
2 2 (0.9999999999999999+3.469446951953614e-17j)
3 3 (0.9999999999999879+8.440513912799652e-17j)
4 4 (0.9999999999998301+1.3424930112894418e-15j)
4 202 (-1.149046689875945e-06-8.189271802516217e-08j)
4 203 (-1.6578461864173965e-06-1.2556979400595588e-07j)
5 5 (0.9999999999999997-2.0816681711721685e-17j)
6 6 (1+6.938893903907228e-18j)
7 7 (0.9999999999999997+2.0383000842727483e-17j)
8 8 (1.0000000000002336-5.024843388601319e-15j)
9 9 (1.0000000000000278+2.9238764187589084e-15j)
10 10 (1+4.83554168928535e-17j)
11 11 (0.999999999999892-4.02325742165921e-15j)
11 202 (-1.0195359813347353e-06-7.045657415065929e-08j)
11 203 (-1.4710021161736827e-06-1.0823295997255252e-07j)
12 12 (0.9999999999999906-3.0704605524789486e-16j)
13 13 (1.0000000000000002+2.1406216643010678e-17j)
14 14 (1+2.7755575615628914e-17j)
15 15 (0.9999999999999999-5.551115123125783e-17j)
16 16 (0.9999999999999946-6.036837696399289

147 147 (0.9999999999999999+6.78168458889683e-17j)
148 148 (0.9999999999999999+6.998525023393931e-17j)
149 149 (1-2.8189256484623115e-18j)
150 150 (1.0000000000003186+2.3852447794681098e-15j)
150 202 (-1.0819943922576619e-06-1.2220277074881965e-07j)
150 203 (-1.5608135455887754e-06-1.833171350691122e-07j)
151 151 (1.0000000000024123-1.256113268954806e-14j)
151 202 (-3.2095454000349247e-06-2.2058046802112276e-07j)
151 203 (-4.630789052892643e-06-3.389614570591668e-07j)
152 152 (0.9999999999990581+1.5565153332897097e-13j)
153 153 (1.0000000000006846+1.8417906089140956e-13j)
154 154 (1.0000000000000002-1.1102230246251565e-16j)
155 155 (1.0000000000000004-9.020562075079397e-17j)
156 156 (1.0000000000000002+1.0408340855860843e-17j)
157 157 (1.0000000000000004-3.0357660829594124e-18j)
158 158 (1.0000000000000804+4.0196253443813834e-15j)
159 159 (1.0000000000008735+3.7319209137895437e-13j)
160 160 (0.9999999999999999-2.439454888092385e-19j)
161 161 (0.9999999999996617+3.946148963152041e-14j)


201 106 (8.895927359730585e-07+2.4346427829740323e-06j)
201 119 (2.0640370734018733e-06-2.590177727641401e-08j)
201 131 (-5.2848524013231655e-06+1.45611963987087e-06j)
201 132 (-5.6803856484827905e-06-6.68863684067161e-07j)
201 142 (2.3301697552355636e-05-6.055988191811442e-06j)
201 143 (-2.8246835923717833e-05+6.443297586817186e-07j)
201 152 (-4.601266986375147e-06+6.977259631770926e-07j)
201 153 (-1.7353084187122291e-06+5.768179754509897e-07j)
201 158 (1.0625936464106247e-06-2.51361627701333e-08j)
201 161 (1.4122902894370608e-06+1.386687245920704e-10j)
201 162 (-3.2774536751651945e-06+1.0525852962379039e-08j)
201 169 (1.6885526378150884e-06-4.439189944936282e-08j)
201 173 (-4.917276864191946e-05+1.1153622685842758e-06j)
201 177 (0.0026331460662132-3.0877710365963897e-05j)
201 178 (0.0015877847464298134-7.238982038683942e-05j)
201 181 (-0.00029454447033551494+1.3067345557959182e-05j)
201 182 (-0.00011537038377823528+1.53622326096756e-05j)
201 186 (4.587967202658821e-05-2.0177165303190

'\np, ML, MU = lu(M, permute_l=False, overwrite_a=False, check_finite=True)\n\nprint(np.allclose(M - p @ ML @ MU, np.zeros((dim, dim),dtype=complex)))\n\nLp = inv(ML) @ LCIS\nRp = RCIS @ inv(MU)\n\nMbo = np.zeros((dim, dim), dtype=complex)\nfor i in range(0, dim):\n    for j in range(0, dim):\n        L = np.conj(Lp[:,i])\n        R = Rp[:,j]\n        o = np.dot(L, R)\n        Mbo[i,j] = o\n        if i==j:\n            print(i,j,o)\n        elif np.isclose(o, 0+0j, atol=1e-6):\n            te = 2+2\n        else:\n            print(i,j,o)\n            \n\nprint(Mbo)\nprint(np.allclose(np.identity(dim, dtype=complex), Mbo, atol=1e-3))\n'

The left and right eigenvectors have been biorthogonalized using the LU decomposition approach described [here](https://joshuagoings.com/blog/page3/).  The above block of python code demonstrates that the left and right eigenvectors are (at least numerically close) to biorthogonal.  The question remains are they still eigenvectors of the Hamiltonian?  We test this here by evaluating

$$ {\bf L}_i^{\dagger} {\bf H} {\bf R}_i = E_i $$
for a given state $i$.

In [16]:
state = 30
Ls = np.conj(LCIS[:, state])
R = RCIS[:, state]

expectation = np.dot(Ls, np.dot(H, R))
print(expectation)
print(cqed_cis_e[state])
print(np.isclose(expectation, cqed_cis_e[state]))

(1.6279844182577594-1.460102604395469e-07j)
(1.6279844182582457-1.4601028481451103e-07j)
True
